<a href="https://colab.research.google.com/github/titiodus/titiodus/blob/master/TitiStageCQuizCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math
from geopy.distance import geodesic
from geopy.distance import lonlat, distance

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [25]:
df.drop('stab', inplace=True, axis=1)
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [26]:
#Check the distribution of target variable
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [27]:
#Check for missing values
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stabf    0
dtype: int64

In [28]:
#more preprocessing
X = df.drop(columns='stabf')
y = df['stabf']

In [29]:
#Splitting the dataset into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
y_train.value_counts()

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [30]:
#standardize the dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_trainscl = scaler.transform(x_train)

In [31]:
x_testscl = scaler.transform(x_test)

In [32]:
x_trainscl1 = pd.DataFrame(x_trainscl, columns = x_train.columns)
x_testscl1 = pd.DataFrame(x_testscl, columns = x_test.columns)

In [33]:
from sklearn.ensemble import RandomForestClassifier
rfcl = RandomForestClassifier(random_state=1)

In [34]:
rfcl.fit(x_trainscl1, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [35]:
rfcl_pred = rfcl.predict(x_testscl1)

In [36]:
y_test.unique()

array(['unstable', 'stable'], dtype=object)

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rfcl_pred, digits=4))

              precision    recall  f1-score   support

      stable     0.9191    0.8778    0.8980       712
    unstable     0.9341    0.9573    0.9456      1288

    accuracy                         0.9290      2000
   macro avg     0.9266    0.9176    0.9218      2000
weighted avg     0.9288    0.9290    0.9286      2000



In [39]:
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix
cf_mat = confusion_matrix(y_test, rfcl_pred, labels=['unstable', 'stable'])
cf_mat

array([[1233,   55],
       [  87,  625]])

In [40]:
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix

In [73]:
accuracy = accuracy_score(y_test, rfcl_pred)
accuracy

0.929

In [42]:
recall = recall_score(y_test, rfcl_pred, pos_label='stable')
print('Recall: {}'.format(round(recall*100), 2))

Recall: 88.0


In [43]:
precision = precision_score(y_test, rfcl_pred, pos_label='stable')
print('Precision: {}'.format(round(precision*100), 2))

Precision: 92.0


In [44]:
f1 = f1_score(y_test, rfcl_pred, pos_label='stable')
print('F1: {}'.format(round(f1*100), 2))

F1: 90.0


In [45]:
from sklearn.ensemble import ExtraTreesClassifier
etcl = ExtraTreesClassifier(random_state=1)

etcl.fit(x_trainscl1, y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [46]:
etcl_pred = etcl.predict(x_testscl1)

In [47]:
print(classification_report(y_test, etcl_pred, digits=4))

              precision    recall  f1-score   support

      stable     0.9410    0.8511    0.8938       712
    unstable     0.9218    0.9705    0.9455      1288

    accuracy                         0.9280      2000
   macro avg     0.9314    0.9108    0.9197      2000
weighted avg     0.9287    0.9280    0.9271      2000



In [48]:
cf_mat1 = confusion_matrix(y_test, etcl_pred)
cf_mat1

array([[ 606,  106],
       [  38, 1250]])

In [49]:
accuracy1 = accuracy_score(y_test, etcl_pred)
print('Accuracy1: {}'.format(round(accuracy1*100), 3))

Accuracy1: 93.0


In [50]:
recall1 = recall_score(y_test, etcl_pred, pos_label='stable')
print('Recall1: {}'.format(round(recall1*100), 2))

Recall1: 85.0


In [51]:
precision1 = precision_score(y_test, etcl_pred, pos_label='stable')
print('Precision1: {}'.format(round(precision1*100), 2))

Precision1: 94.0


In [52]:
f1_1 = f1_score(y_test, etcl_pred, pos_label='stable')
print('F1_1: {}'.format(round(f1_1*100), 2))

F1_1: 89.0


In [53]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=1)
xgb.fit(x_trainscl1, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [54]:
xgb_pred = xgb.predict(x_testscl1)

In [55]:
print(classification_report(y_test, xgb_pred, digits=4))

              precision    recall  f1-score   support

      stable     0.9206    0.8469    0.8822       712
    unstable     0.9190    0.9596    0.9389      1288

    accuracy                         0.9195      2000
   macro avg     0.9198    0.9033    0.9105      2000
weighted avg     0.9195    0.9195    0.9187      2000



In [56]:
cf_mat2 = confusion_matrix(y_test, xgb_pred)
cf_mat2

array([[ 603,  109],
       [  52, 1236]])

In [74]:
accuracy2 = accuracy_score(y_test, xgb_pred)
accuracy2

0.9195

In [58]:
recall2 = recall_score(y_test, xgb_pred, pos_label='stable')
print('Recall2: {}'.format(round(recall2*100), 2))

Recall2: 85.0


In [59]:
precision2 = precision_score(y_test, xgb_pred, pos_label='stable')
print('Precision2: {}'.format(round(precision2*100), 2))

Precision2: 92.0


In [60]:
f1_2 = f1_score(y_test, xgb_pred, pos_label='stable')
print('F1_2: {}'.format(round(f1_2*100), 2))

F1_2: 88.0


In [61]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=1)
lgbm.fit(x_trainscl1, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [62]:
lgbm_pred = lgbm.predict(x_testscl1)

In [63]:
print(classification_report(y_test, lgbm_pred, digits=4))

              precision    recall  f1-score   support

      stable     0.9297    0.8919    0.9104       712
    unstable     0.9415    0.9627    0.9520      1288

    accuracy                         0.9375      2000
   macro avg     0.9356    0.9273    0.9312      2000
weighted avg     0.9373    0.9375    0.9372      2000



In [64]:
cf_mat3 = confusion_matrix(y_test, lgbm_pred)
cf_mat3

array([[ 635,   77],
       [  48, 1240]])

In [75]:
accuracy3 = accuracy_score(y_test, lgbm_pred)
accuracy3

0.9375

In [66]:
recall3 = recall_score(y_test, lgbm_pred, pos_label='stable')
print('Recall3: {}'.format(round(recall3*100), 2))


Recall3: 89.0


In [67]:
precision3 = precision_score(y_test, lgbm_pred, pos_label='stable')
print('Precision3: {}'.format(round(precision3*100), 2))

Precision3: 93.0


In [68]:
f1_3 = f1_score(y_test, lgbm_pred, pos_label='stable')
print('F1_3: {}'.format(round(f1_3*100), 2))

F1_3: 91.0


In [69]:
from sklearn.model_selection import RandomizedSearchCV

In [70]:
n_estimators =  [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

In [71]:
hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

In [80]:
rdmscv = RandomizedSearchCV(n_estimators,hyperparameter_grid, n_iter=10, scoring='accuracy', n_jobs=-1, cv=5, verbose=1, random_state=1)
rdmscv

RandomizedSearchCV(cv=5, error_score=nan, estimator=[50, 100, 300, 500, 1000],
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 3, 5, 7, 9],
                                        'n_estimators': [50, 100, 300, 500,
                                                         1000]},
                   pre_dispatch='2*n_jobs', random_state=1, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=1)